# Advanced Data Wrangling with Pandas

In [ ]:
import pandas as pd
import numpy as np

## Formas não usuais de se ler um dataset

Você não precisa que o arquivo com os seus dados esteja no seu disco local, o pandas está preparado para adquirir arquivos via http, s3, gs...

In [ ]:
diamonds = pd.read_csv("https://raw.githubusercontent.com/mwaskom/seaborn-data/master/diamonds.csv")
diamonds.head()

Você também pode crawlear uma tabela de uma página da internet de forma simples

In [ ]:
clarity = pd.read_html("https://www.brilliantearth.com/diamond-clarity/")

clarity

In [ ]:
clarity = clarity[0]
clarity

In [ ]:
clarity.columns = ['clarity', 'clarity_description']
clarity

## Como explodir a coluna de um dataframe

In [ ]:
clarity['clarity'] = clarity['clarity'].str.split()
clarity

In [ ]:
type(clarity.loc[0, 'clarity'])

In [ ]:
clarity = clarity.explode("clarity")
clarity

## Como validar o merge

Esse parametro serve para validar a relação entre as duas tabelas que você está juntando. Por exemplo, se a relação é 1 para 1, 1 para muitos, muitos para 1 ou muitos para muitos.

In [ ]:
diamonds.merge(clarity, on='clarity', validate="m:1")

In [ ]:
clarity_with_problem = clarity.append(pd.Series({"clarity": "SI2", "clarity_description": "slightly included"}), ignore_index=True)
clarity_with_problem

In [ ]:
diamonds.merge(clarity_with_problem, on='clarity', validate="m:1")

In [ ]:
diamonds.merge(clarity_with_problem, on='clarity')

### Por que isso é importante?

O que aconteceria seu tivesse keys duplicadas no meu depara. Ele duplicou as minhas linhas que tinham a key duplicada, o dataset foi de 53,940 linhas para 63,134 linhas

## Como usar o método `.assign`

Para adicionar ou modificar colunas do dataframe. Você pode passar como argumento uma constante para a coluna ou um função que tenha como input um `pd.DataFrame` e output uma `pd.Series`.

In [ ]:
diamonds.assign(foo="bar", bar="foo")

In [ ]:
diamonds.assign(volume=lambda df: df['x'] * df['y'] * df['z'])

In [ ]:
def calculate_volume(df):
    return df['x'] * df['y'] * df['z']


diamonds.assign(volume=calculate_volume)

In [ ]:
diamonds['volume'] = diamonds['x'] * diamonds['y'] * diamonds['z']
diamonds

## Como usar o método `.query`
Para filtrar. Tende a ser util quando você quer filtrar o dataframe baseado em algum estado intermediário

In [ ]:
diamonds = pd.read_csv("https://raw.githubusercontent.com/mwaskom/seaborn-data/master/diamonds.csv")
diamonds.head()

In [ ]:
diamonds.describe()

In [ ]:
diamonds[(diamonds['x'] == 0) | (diamonds['y'] == 0) | (diamonds['z'] == 0)]

In [ ]:
diamonds.query("x == 0 | y == 0 | z == 0")


In [ ]:
x = diamonds \
    .assign(volume=lambda df: df['x'] * df['y'] * df['z'])

x = x[x['volume'] > 0]

In [ ]:
diamonds = diamonds \
    .assign(volume=lambda df: df['x'] * df['y'] * df['z']) \
    .query("volume > 0")

diamonds

Você também pode usar variáveis externas ao dataframe dentro da sua query, basta usar @ como marcador.

In [ ]:
selected_cut = "Premium"
diamonds.query("cut == @selected_cut")

Quase qualquer string que seria um código python válido, vai ser uma query valida

In [ ]:
diamonds.query("clarity.str.startswith('SI')")

Porém o parser do pandas tem algumas particularidades, como o `==` que também pode ser um `isin`

In [ ]:
diamonds.query("color == ['E', 'J']")

In [ ]:
diamonds = diamonds.query("x != 0 & y != 0 & z != 0")

Exemplo de que precisamos do estado intermediário para fazer um filtro. Você cria uma nova coluna e quer filtrar baseado nela sem precisar salvar esse resultado em uma variável intermerdiária

## Como usar o método `.loc` e `.iloc`
Uma das desvantagens do `.query` é que fica mais difícil fazer análise estática do código, os editores geralmente não suportam syntax highlighting. Um jeito de solucionar esse problemas é usando o `.loc` ou `.iloc`, que além de aceitarem mascaras, eles aceitam funções também.

In [ ]:
diamonds.loc[[0, 1, 2], ['clarity', 'depth']]

In [ ]:
diamonds.iloc[[0, 1, 2], [3, 4]]

In [ ]:
diamonds.sort_values("depth")

In [ ]:
diamonds.sort_values("depth").loc[[0, 1, 2]]

In [ ]:
diamonds.sort_values("depth").iloc[[0, 1, 2]]

In [ ]:
diamonds.loc[diamonds["price"] > 6000]

In [ ]:
diamonds["price"] > 6000

In [ ]:
diamonds.loc[lambda x: x['price'] > 6000]

In [ ]:
diamonds[diamonds['price'] > 10000]['price'] = 10000

In [ ]:
diamonds.query("price > 10000")

In [ ]:
diamonds.loc[diamonds['price'] > 10000, 'price'] = 10000

In [ ]:
diamonds.query("price > 10000")

## O que o `.groupby(...) retorna`

In [ ]:
diamonds = pd.read_csv("https://raw.githubusercontent.com/mwaskom/seaborn-data/master/diamonds.csv") \
    .assign(volume=lambda x: x['x'] * x['y'] * x['z']) \
    .query("volume > 0")
diamonds.head()

In [ ]:
grouped_diamonds = diamonds.groupby("cut")
grouped_diamonds

In [ ]:
list(grouped_diamonds)

## Os N formatos de agregação do pandas

A função `.agg` é um *alias* da função `.aggregate`, então elas tem o mesmo resultado.

O Pandas tem algumas funções padrão que permitem que você passe só o nome delas, ao invés do *callable*:
* "all"
* "any"
* "count"
* "first"
* "idxmax"
* "idxmin"
* "last"
* "mad"
* "max"
* "mean"
* "median"
* "min"
* "nunique"
* "prod"
* "sem"
* "size"
* "skew"
* "std"
* "sum"
* "var"

Você pode passar uma lista de callable e o pandas vai aplicar todas as funções para todas as colunas. Faz sentido se são muitas funções e poucas colunas. Um problema é que ele vai nomear as novas colunas com base na coluna anterior e na função, quando você usa uma lambda isso causa um problema.

In [ ]:
diamonds.groupby('clarity').agg(['mean', 'sum', np.max, lambda x: x.min()])

Você também pode passar um dicionário de listas, assim você pode escolher qual função será aplicada em cada coluna, você ainda tem o problema de nome das novas colunas ao usar uma função anônima.

In [ ]:
diamonds.groupby('clarity').agg({"x": 'mean', 'price': [np.max, 'max', max, lambda x: x.max()]})

A terceira opção é o NamedAgg foi lançada recentemente. Ela resolve o problema de nomes de colunas. Você passa como parâmetro uma tupla para cada agregação que você quer. O primeiro elemento é o nome da coluna e o segundo é a função.

\* *O Dask ainda não aceita esse tipo de agregação*

In [ ]:
diamonds.groupby('clarity').agg(max_price=('price', 'max'), total_cost=('price', lambda x: x.sum()))

## `.groupby(...).apply(...)`

Um problema comum a todas essas abordagens é que você não consegue fazer uma agregação que depende de duas colunas. Para a maior parte dos casos existe uma forma razoável de resolver esse problema criando uma nova coluna e aplicando a agregação nela. Porém, se isso não foi possível, dá para usar o `.groupby(...).apply()`.

In [ ]:
# Nesse caso ao invés da função de agregação receber a pd.Series relativa ao grupo,
# ela vai receber o subset do grupo. Aqui vamos printar cada grupo do df de forma 
# separada

diamonds.groupby('cut').apply(lambda x: print(x.head().to_string() + "\n"))

Esse formato de agregação introduz algumas complexidades, porque sua função pode retornar tanto um pd.DataFrame, pd.Series ou um escalar. O pandas vai tentar fazer um broadcasting do que você retorna para algo que ele acha que faz sentido. Exemplos:

Se você retornar um escalar, o apply vai retornar uma `pd.Series` em que cada elemento corresponde a um grupo do .groupby

In [ ]:
# Retornando um escalar
def returning_scalar(df: pd.DataFrame) -> float:
    return (df["x"] * df["y"] * df['z']).mean()


diamonds.groupby("cut").apply(returning_scalar)

Se você retornar uma `pd.Series` nomeada, o apply vai retornar um `pd.DataFrame` em que cada linha corresponde a um grupo do `.groupby` e cada coluna corresponde a uma key do pd.Series que você retorna na sua função de agregação

In [ ]:
def returning_named_series(df: pd.DataFrame) -> pd.Series:
    volume = (df["x"] * df["y"] * df['z'])
    price_to_volume = df['price'] / volume
    return pd.Series({"mean_volume": volume.mean(), "mean_price_to_volume": price_to_volume.mean()})


diamonds.groupby("cut").apply(returning_named_series)

Se você retornar um `pd.DataFrame`, o apply vai retornar uma concatenação dos desses `pd.DataFrame`

In [ ]:
def returning_dataframe(df: pd.DataFrame) -> pd.DataFrame:
    return df[df['volume'] >= df['volume'].median()]
    

diamonds.groupby("cut").apply(returning_dataframe)

Se você retornar uma `pd.Series` não nomeada, o apply vai retornar uma `pd.Series` que é uma concatenação das `pd.Series` que você retorna da sua função

In [ ]:
def returning_unnamed_series(df: pd.DataFrame) -> pd.Series:
    return df.loc[df['volume'] >= df['volume'].median(), 'volume']


diamonds.groupby("cut").apply(returning_unnamed_series)

De forma resumida, o `.groupby(...).apply(...)` é extremamente flexível, ele consegue filtrar, agregar e tranformar. Mas é mais complicado de usar e é bem lento se comparado aos outros métodos de agregação. Só use se necessário.

| Saída da Função       | Saída do apply                                                                                                                                                     |
|-----------------------|--------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| Escalar               | Uma pd.Series em que cada elemento corresponde a um grupo do .groupby                                                                                              |
| pd.Series nomeada     | Um pd.DataFrame em que cada linha corresponde a um grupo do .groupby e cada coluna corresponde a uma key do pd.Series que você retorna na sua função de agregação  |
| pd.Series não nomeada | Uma `pd.Series` que é uma concatenação das `pd.Series` que você retorna da sua função                                                                              |
| pd.DataFrame          | Uma concatenação dos desses `pd.DataFrame`                                                                                                                         |

## Como usar o método `.pipe`

O `.pipe` aplica uma função ao dataframe

In [ ]:
def change_basis(df: pd.DataFrame, factor=10):
    df[['x', 'y', 'z']] = df[['x', 'y', 'z']] * factor
    return df


diamonds.pipe(change_basis)

Nós não atribuimos o resultado da nossa operação a nenhuma variável, então teoricamente se rodarmos de novo, o resultado vai ser o mesmo.

In [ ]:
diamonds.pipe(change_basis)

Isso acontece porque a sua função está alterando o `pd.DataFrame` original ao invés de criar uma cópia, isso é um pouco contra intuitivo porque o Pandas por padrão faz as suas operações em copias da tabela. Para evitar isso podemos fazer uma cópia do dataframe manualmente

In [ ]:
diamonds = pd.read_csv("https://raw.githubusercontent.com/mwaskom/seaborn-data/master/diamonds.csv")

In [ ]:
def change_basis(df: pd.DataFrame, factor=10):
    df = df.copy()
    df[['x', 'y', 'z']] = df[['x', 'y', 'z']] * factor
    return df


diamonds.pipe(change_basis, factor=10)

In [ ]:
diamonds

## Como combinar o `.assign`, `.pipe`, `.query` e `.loc` para um Pandas mais idiomático

Os métodos mais importantes para *Method Chaining* são
* `.assign`
* `.query`
* `.loc`
* `.pipe`

In [ ]:
diamonds = pd.read_csv("https://raw.githubusercontent.com/mwaskom/seaborn-data/master/diamonds.csv")
diamonds.head()

In [ ]:
diamonds_cp = diamonds.copy()
diamonds_cp[['x', 'y', 'z']] = diamonds_cp[['x', 'y', 'z']] * 10
diamonds_cp['volume'] = diamonds_cp['x'] * diamonds_cp['y'] * diamonds_cp['z']
diamonds_cp = diamonds_cp[diamonds_cp['volume'] > 0]
diamonds_cp = pd.merge(diamonds_cp, clarity, on='clarity', how='left')

diamonds_cp

In [ ]:
def change_basis(df: pd.DataFrame, factor=10):
    df = df.copy()
    df[['x', 'y', 'z']] = df[['x', 'y', 'z']] * factor
    return df


diamonds \
    .copy() \
    .pipe(change_basis, factor=10) \
    .assign(volume=lambda df: df['x'] * df['y'] * df['z']) \
    .query("volume > 0") \
    .merge(clarity, on='clarity', how='left')


Um problema que pode acontecer quando você usa o method chaining é você acabar com um bloco gigantesco que é impossível de debugar, uma boa prática é quebrar seus blocos por objetivos

In [ ]:
diamonds.query("x > 0").groupby(["cut", "color"]).size()

## Como mandar um dataframe para a sua clipboard
Geralmente isso não é uma boa pratica, mas as vezes é útil para enviar uma parte do dado por mensagem ou para colar em alguma planilha.

In [ ]:
df = pd.DataFrame({'a':list('abc'), 'b':np.random.randn(3)})

In [ ]:
df

In [ ]:
df.to_clipboard()

In [ ]:
df.to_csv("df.csv")

Você também pode ler da sua *clipboard* com `pd.read_clipboard(...)`. O que é uma prática pior ainda, mas em alguns casos pode ser útil.

## Recursos
https://pandas.pydata.org/docs/user_guide/cookbook.html

https://tomaugspurger.github.io/modern-1-intro.html